In [14]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does n ot exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)


enter file name: Reading data from file 'lab1-big.mat'
Datafile: 'lab1-big.mat'
Rows: 400 cols: 1400
c: [-6.94147382 -5.35053556 -6.4511742  ... -0.         -0.
 -0.        ]
b: [33.98276893 48.39735208 14.29640933 29.75163284 35.6931775  45.23431514
  3.1345862  32.99022731 16.75032844 22.77003802 46.02908417 10.81771201
 13.28583479 45.02532908 40.70584683 43.16763493 18.44033237 26.66606574
 26.56684149  8.90490509 14.73977827 31.45342121 37.65034943 39.42283275
 29.55003676 47.46774433 40.3788866  47.94702697 45.68236037 33.43073127
 20.30037994 38.48565912 28.47279684 42.29650355 27.33517344 22.26000366
 23.88157615 27.6503788  19.91644186 35.63833494 23.49541517 37.44270701
 49.57669056 35.43824208 10.53467591  6.29807283  1.70997523 39.55374383
 29.77253084 36.92575977 11.24438297 34.34458712 29.47570106 48.10763551
 45.0299887  19.0200195  19.46780012 45.31661086 36.27865114 35.28964449
 13.05493781 14.33969772  7.2995282  33.1705332  47.15141985 23.91333516
 11.42402371  3.566

In [15]:
from ctypes import sizeof
import numpy as np
import time
import sys

#Simplex i matrisform s.112 i Optimering av Kaj Holmberg

rc_min = np.nanmin(c)

t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))
i = 0
# Create nix
nix=np.setdiff1d(range(n), bix) 
#Steg 1, skaffa en tillåten bas
B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]

# Start iterations
iter = 0
while iter >= 0:
    iter+=1            

    #Steg 2, invers av B
    inverseB = np.linalg.inv(B)
    #Steg 3, beräkna högerledet
    bHatt = np.matmul(inverseB, b)
    
    #Steg 4, beräkna skuggpriset
    y = np.transpose(np.matmul(np.transpose(cB), inverseB))
    #Steg 5, beräkna målfunktionsvärdet
    z= np.matmul(np.transpose(b),y)

    #Ste 6, beräkna reducerad kostnad
    cNhatt = (cN - np.matmul(np.transpose(N),y))

    #Steg 8, välj inkommande variabel
    inkvarPlats = np.argmin(cNhatt)
    rc_min = cNhatt[inkvarPlats]
    inkvar = nix[inkvarPlats]

    #Steg 7, om optimum funnat stoppa
    if rc_min >= -1.0E-10:
        print ('Ready')
        iter=-1
 
        #Skapa en array med lika många platcer som antal variabler i problemet
        x = np.full([np.size(c)], 0.00)
        #Starta en python sorting som sorterar efter storleken av bix
        p = bix.argsort()
        t = bHatt[p]
        x[:len(t)] += t

        #Steg 5, beräkna funktionsvärdet
        z = np.matmul(np.transpose(b),y)
        print("funktionsvärde: ",z)

        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        #Steg 9, beräkna inkommande kolumn
        ak = np.matmul(inverseB, N[:, inkvarPlats])
        if max(ak) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            #Steg 11, välj utgående variabel
            alkHatt = np.divide(bHatt,ak)
            utgvarPlats = np.where(alkHatt > 0, alkHatt, np.inf).argmin()
            utgvar = bix[utgvarPlats]

            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # -------
            
           #Steg 12, byt till en ny bas
            bix[utgvarPlats] = nix[inkvarPlats]

            B = A[:,bix]
            N = A[:,nix]
            cB = c[bix]
            cN = c[nix]
            

   



elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

Rows: 400 cols: 1400
 Iter: 1 z: 0.0 rc: -9.97648483606823 ink: 329 utg: 1131
 Iter: 2 z: -5.460434400842103 rc: -12.533800057293249 ink: 435 utg: 1362
 Iter: 3 z: -8.126304582614104 rc: -10.399435113316942 ink: 522 utg: 1124
 Iter: 4 z: -15.28110835360965 rc: -22.396403076973428 ink: 313 utg: 435
 Iter: 5 z: -23.70733794423675 rc: -24.936765046919003 ink: 617 utg: 1193
 Iter: 6 z: -26.136681130677623 rc: -15.537052178213887 ink: 189 utg: 1237
 Iter: 7 z: -32.47269047373251 rc: -39.547600415913564 ink: 691 utg: 329
 Iter: 8 z: -34.79572311992543 rc: -10.466789327820564 ink: 719 utg: 1194
 Iter: 9 z: -40.46982626897547 rc: -13.375312721481144 ink: 928 utg: 691
 Iter: 10 z: -41.61424160562507 rc: -9.04476836146339 ink: 177 utg: 1344
 Iter: 11 z: -43.061852202638775 rc: -15.227757318809639 ink: 206 utg: 1111
 Iter: 12 z: -43.481072374423604 rc: -12.077340147390245 ink: 572 utg: 1173
 Iter: 13 z: -43.49453272650556 rc: -7.5391236474228105 ink: 677 utg: 1389
 Iter: 14 z: -43.712852924838884